#Importing all libraries, google disk for loading dataset


In [ ]:
!pip install optuna
import optuna
import torch
import numpy as np
from torchvision import transforms, datasets
import os
from tqdm import tqdm_notebook
import torchvision
from torch import nn
from torch.utils.data import Dataset, ConcatDataset,  SubsetRandomSampler
import matplotlib.pyplot as plt
from torchvision.io import read_image
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import datasets, models
from torch.optim import lr_scheduler

from optuna.trial import TrialState
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(69)

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
#device = torch.device("cpu")
# device
# device = "mps" if torch.backends.mps.is_available() else "cpu"
print(f"Using device: {device}")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Different augmentations and transformations, loading dataset


In [ ]:

transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
])

transform2 = transforms.Compose(

    [
     transforms.ToTensor(),
     #transforms.Normalize((0.5, 0.5, 0.5), (0.25, 0.25, 0.25))
    ]
)

transformImageNet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transformResNet152 = transforms.Compose([
    transforms.Resize(256),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transformGoogle = transforms.Compose([
    transforms.Resize((299, 299)),  # InceptionV3 input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])



In [ ]:
import os
from torchvision import transforms
from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, folder_path, transform=None, fake_label=0, real_label=1):
        self.folder_path = folder_path
        self.image_paths = [os.path.join(folder_path, img) for img in os.listdir(folder_path) if img.endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
        self.transform = transform
        self.fake_label = fake_label
        self.real_label = real_label

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        # Assign labels based on folder path
        if "fake" in self.folder_path:
            label = self.fake_label
        else:
            label = self.real_label

        return image, label



In [ ]:
training_folder = '/content/drive/MyDrive/project DeepFakes/data/training'

trainset = datasets.ImageFolder(root=training_folder, transform=transform2)
# fake_aug = CustomDataset('/content/drive/MyDrive/project DeepFakes/data/training/fake', transform=transform)
# real_aug = CustomDataset('/content/drive/MyDrive/project DeepFakes/data/training/real', transform=transform)
# combined_dataset = ConcatDataset([trainset, fake_aug, real_aug])

# total_samples = len(combined_dataset)
# num_samples = int(0.8 * total_samples)

# sampler = SubsetRandomSampler(torch.randperm(total_samples)[:num_samples])

# trainloader = DataLoader(combined_dataset, batch_size=16, sampler=sampler)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = 16, shuffle = True)

test_folder = '/content/drive/MyDrive/project DeepFakes/data/validation'
testset = datasets.ImageFolder(root=test_folder, transform=transform2)
testloader = torch.utils.data.DataLoader(testset, batch_size = 16, shuffle = False)

In [ ]:
for images, labels in trainloader:
    print("Batch Shape:", images)
    print("Labels Shape:", labels)
    break
# print(trainloader.dataset.cummulative_sizes)

#First Model with parameters that were get from optuna

In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels = 32, kernel_size = 5, stride = 3, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels= 32 , out_channels = 64, kernel_size = 5, stride = 3, padding=2)
        self.bn2 = nn.BatchNorm2d(64)
        # self.dropout = nn.Dropout(p=0.2)
        self.fc1 = nn.Linear(17 * 17 * 64, 1000)
        self.fc2 = nn.Linear(1000, 1)
        #self.fc3 = nn.Linear(30, 1)
        #self.fc2_2 = nn.Linear(1000, 1)


    def forward(self, x):
        x = self.bn1(self.pool(F.relu(self.conv1(x))))
        x = self.bn2(self.pool(F.relu(self.conv2(x))))
        #print(x.shape)
        x = x.view(-1, 17 * 17 * 64)
        # x = self.dropout(x)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc2(x))
        return x


In [ ]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.skip_connection1 = nn.Conv2d(in_channels=3, out_channels = 64, kernel_size = 7, stride = 1, padding=1)

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels = 32, kernel_size = 7, stride = 1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels= 32 , out_channels = 64, kernel_size = 3, stride = 1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels= 64 , out_channels = 64, kernel_size = 5, stride = 1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(in_channels= 64 , out_channels = 64, kernel_size = 3, stride = 2, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv5 = nn.Conv2d(in_channels= 64 , out_channels = 128, kernel_size = 3, stride = 2, padding=1)
        self.bn3 = nn.BatchNorm2d(128)

        self.dropout = nn.Dropout(p=0.5)
        self.fc1 = nn.Linear(4 * 4 * 128, 512)
        self.fc2 = nn.Linear(512, 1)
        #self.fc3 = nn.Linear(30, 1)
        #self.fc2_2 = nn.Linear(1000, 1)


    def forward(self, x):
        x = self.bn1(self.pool(F.relu(self.conv1(x))))
        x = self.bn2(self.pool(F.relu(self.conv2(x))))
        x = (self.pool(F.relu(self.conv3(x))))
        x = self.bn2(self.pool(F.relu(self.conv4(x))))
        x = self.bn3(self.pool(F.relu(self.conv5(x))))
        #print(x.shape)
        x = x.view(-1, 4 * 4 * 128)

        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        #x = F.relu(self.fc2(x))
        x = F.sigmoid(self.fc2(x))
        return x

In [ ]:
net = models.vgg11(pretrained = True)
print(net)

In [ ]:
batch = []
net = ConvNet().to(device)

batch = next(iter(trainloader))

net.forward(torch.FloatTensor(batch[0]).to(device))

In [ ]:
loss_fn = torch.nn.BCEWithLogitsLoss()

learning_rate = 0.00010246617213508516 #for adam
#learning_rate = 1.71903735673968e-05 #for RMS

optimizer = optim.Adam(net.parameters(), lr=learning_rate)
#optimizer = torch.optim.RMSprop(net.parameters(), lr = learning_rate)
losses = []

In [ ]:
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(10, 1, 1)

for epoch in tqdm_notebook(range(10)):

    running_loss = 0.0
    for i, batch in enumerate(tqdm_notebook(trainloader)):

        X_batch, y_batch = batch
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        y_batch = y_batch.float()

        optimizer.zero_grad()

        y_pred = net(X_batch)

        loss = loss_fn(y_pred, y_batch.unsqueeze(1))

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 50 == 49:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            losses.append(running_loss / 50)
            running_loss = 0.0
ax.plot(np.arange(len(losses)), losses)
plt.show()

In [ ]:
class_correct = [0,0]
class_total = [0,0]
classes = ['Real', 'Fake']
net.eval()



with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)

        y_pred = net(images)

        predicted = torch.squeeze(torch.round(y_pred))

        c = predicted.cpu().detach() == labels

        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

    for i in range(2):
        print("Accuracy of %5s : %2d %%" % (classes[i], 100 * class_correct[i] / class_total[i]))
    print(class_total)
    print(class_correct)
    print('Total Accuracy is: ', sum(class_correct) / sum(class_total))

#Optuna model for searching best number of layers and neurons in them

In [ ]:
class OptunaModel(nn.Module):
    def __init__(self, input_size, output_size, num_conv_layers, trial):



        super(OptunaModel, self).__init__()

        in_channels = input_size
        self.conv_layers = nn.ModuleList()
        for i in range(num_conv_layers):
            num_neurons = trial.suggest_int(f'num_neurons_layer_{_}', 128 // (1.5 ** i ), 512 // (1.5 ** i ))
            self.conv_layers.append(nn.Conv2d(input_size, num_neurons, kernel_size = 5, stride = 3, padding = 2 ))
            self.conv_layers.append(nn.ReLU())
            self.conv_layers.append(nn.MaxPool2d(kernel_size=2, stride=2))

            in_channels = num_neurons

        self.flatten = nn.Flatten()
        self.l_layers = nn.ModuleList()

        self.output_layer = nn.Linear(in_channels, output_size)
        self.sigmoid = nn.Sigmoid()


        for i in range(2):
            num_neurons = trial.suggest_int(f'num_neurons_l_{i}', 64 // (1.5 ** i ), 512 // (1.5 ** i ))
            self.l_layers.append(nn.Linear(input_size, num_neurons))
            self.l_layers.append(nn.ReLU())
            in_channels = num_neurons

        self.output_layer = nn.Linear(in_channels, output_size)


    def forward(self, x):
        for layer in self.conv_layers:
            x = layer(x)
        x = self.flatten(x)

        for layer in self.l_layers:
            x = layer(x)

        x = self.output_layer(x)
        x = self.sigmoid(x)
        return x

def objective(trial):

    input_size = 3
    output_size = 1
    num_conv_layers = trial.suggest_int('num_layers', 2, 5)
    model = OptunaModel(input_size, output_size, num_conv_layers, trial).to(device)

    loss_fn = torch.nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr= 1e-4)


    for epoch in tqdm_notebook(range(10)):

        running_loss = 0.0
        for i, batch in enumerate(tqdm_notebook(trainloader)):

            X_batch, y_batch = batch
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            y_batch = y_batch.float()

            optimizer.zero_grad()

            y_pred = model(X_batch)

            loss = loss_fn(y_pred, y_batch.unsqueeze(1))

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 50 == 49:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 50))
                losses.append(running_loss)
                running_loss = 0.0

    class_correct = [0,0]
    class_total = [0,0]
    classes = ['Real', 'Fake']

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)

            y_pred = model(images)

            predicted = torch.squeeze(torch.round(y_pred))

            c = predicted.cpu().detach() == labels

            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    return sum(class_correct) / sum(class_total)



In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials = 3)

best_trial = study.best_trial
best_trial.value

In [ ]:
best_trial.params

In [ ]:
def train_model(model, loss_fn, optimizer, scheduler = None, num_epochs = 10, effnet = False):

    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(10, 1, 1)

    best_model_wts = model.state_dict()
    best_acc = 0.0
    acc = []



    for epoch in tqdm_notebook(range(num_epochs)):

        if scheduler is not None:
          scheduler.step()
        running_loss = 0.0
        model.train()

        for i, batch in enumerate(tqdm_notebook(trainloader)):

            X_batch, y_batch = batch
            y_batch = y_batch.float()
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()

            y_pred = model(X_batch)

            loss = loss_fn(y_pred, y_batch.unsqueeze(1))

            if not effnet:
              loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 25 == 24:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 25))
                #losses.append(running_loss / 50)
                running_loss = 0.0


        class_correct = [0,0]
        class_total = [0,0]
        classes = ['Real', 'Fake']
        model.eval()

        with torch.no_grad():
            for data in testloader:
                images, labels = data
                images = images.to(device)

                y_pred = model(images)

                predicted = torch.squeeze(torch.round(y_pred))

                c = predicted.cpu().detach() == labels

                for i in range(len(labels)):
                    label = labels[i]
                    class_correct[label] += c[i].item()
                    class_total[label] += 1


        for i in range(2):
            print("Accuracy of %5s : %2d %%" % (classes[i], 100 * class_correct[i] / class_total[i]))
        print(class_total)
        print(class_correct)
        print('Total Accuracy is: ', sum(class_correct) / sum(class_total))
        accuracy = sum(class_correct) / sum(class_total)
        acc.append( sum(class_correct) / sum(class_total))
        if accuracy > best_acc and accuracy > 0.63:
            best_acc = accuracy
            best_model_wts = model.state_dict()
            print("New Best Score!!! ")
            torch.save(model.state_dict(), 'own_' + str(accuracy) + '_on20epoch_adam(0.0001), batch_size = 16 Imagenet.pth')




    model.load_state_dict(best_model_wts)
    ax.plot(np.arange(len(acc)), acc)
    plt.show()
    return model, acc




In [ ]:
batch = []
net = ConvNet().to(device)

loss_fn = torch.nn.BCEWithLogitsLoss()

#learning_rate = 0.001 #for adam
learning_rate = 0.00010246617213508516 #for adam
#learning_rate = 1.71903735673968e-05 #for RMS

optimizer = optim.Adam(net.parameters(), lr=learning_rate)

#optimizer = torch.optim.RMSprop(net.parameters(), lr = learning_rate)
losses = []

net, acc = train_model(net, loss_fn, optimizer, num_epochs = 10)

In [ ]:
class_correct = [0,0]
class_total = [0,0]
classes = ['Real', 'Fake']

net.eval()

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)

        y_pred = net(images)

        predicted = torch.squeeze(torch.round(y_pred))

        c = predicted.cpu().detach() == labels

        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(2):
    print("Accuracy of %5s : %2d %%" % (classes[i], 100 * class_correct[i] / class_total[i]))
print(class_total)
print(class_correct)
print(sum(class_correct) / sum(class_total))

#Part where big models are retrained

In [ ]:
effnet = models.efficientnet_b1(pretrained = True)

# for param in model_vgg.features[:-1].parameters():
#     param.requires_grad = False

effnet.classifier[1] = nn.Sequential(
    nn.Linear(1280, 1),
    nn.Sigmoid()
)

# param_groups = [
#     {'params': model_vgg.features.parameters(), 'lr': 0.00001},
#     {'params': model_vgg.classifier.parameters(), 'lr': 0.001}
# ]

effnet = effnet.to(device)

loss_fn2 = torch.nn.BCEWithLogitsLoss()

optimizer = optim.Adam(effnet.parameters(), lr=0.0001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=1)

In [ ]:
effnet, acc = train_model(effnet, loss_fn2, optimizer, exp_lr_scheduler, num_epochs = 15)

In [ ]:
resnet = models.resnet34(pretrained = True) #best works without any staff. Straightforward model retraining

print(resnet)
# for param in resnet.parameters():
#     param.requires_grad = False

# for param in resnet.layer4.parameters():
#     param.requires_grad = True
# for param in resnet.layer3.parameters():
#     param.requires_grad = True



resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 1),
    nn.Sigmoid()
)

# param_groups = [
#     {'params': resnet.layer1.parameters(), 'lr': 0.000001},
#     {'params': resnet.layer2.parameters(), 'lr': 0.00001},
#     {'params': resnet.layer3.parameters(), 'lr': 0.0001},
#     {'params': resnet.layer4.parameters(), 'lr': 0.001},
#     {'params': resnet.fc.parameters(), 'lr': 0.001}
# ]

resnet = resnet.to(device)

loss_fn2 = torch.nn.BCEWithLogitsLoss()

optimizer3 = optim.Adam(resnet.parameters(), lr = 0.0001)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer3, step_size=10, gamma=1)

In [ ]:
resnet, acc = train_model(resnet, loss_fn2, optimizer3, exp_lr_scheduler, num_epochs = 15)

In [ ]:

class_correct = [0,0]
class_total = [0,0]
classes = ['Real', 'Fake']
resnet.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)

        if len(labels) == 2:
            continue
        y_pred = resnet(images)


        predicted = torch.squeeze(torch.round(y_pred))
        # print(predicted)
        # print(labels)
        c = predicted.cpu().detach() == labels


        #print(labels)
        for i in range(len(labels)):

            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print(class_correct, class_total)
print(sum(class_correct) / sum(class_total))


In [ ]:
gnet = models.googlenet(pretrained = True)

gnet.fc = nn.Sequential(
    nn.Linear(1024, 1, bias = True),
    nn.Sigmoid()
)

# for param in gnet.parameters():
#     param.requires_grad = False

gnet = gnet.to(device)
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(gnet.parameters(), lr=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=1.0)


In [ ]:
gnet, acc = train_model(gnet, loss_fn, optimizer, scheduler, num_epochs = 15, effnet = False)

#Downloading best retrained models in terms of accuracy (> 0.8) and combining them together



In [ ]:
class MyModule(nn.Module):
  def __init__(self):
    super(MyModule, self).__init__()
    self.conv_skip_con = nn.Conv2d(3, 128, kernel_size = 105, stride = 105, padding = 1)
    self.conv1 = nn.Conv2d(3, 32, kernel_size = 5, stride = 3, padding = 1)
    self.bn1 = nn.BatchNorm2d(32)
    self.pool = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.conv2 = nn.Conv2d(32, 64, kernel_size = 5, stride = 3, padding = 1)
    self.bn2 = nn.BatchNorm2d(64)

    self.conv3 = nn.Conv2d(64, 128, kernel_size = 5, stride = 3, padding = 1)
    self.bn3 = nn.BatchNorm2d(128)
    self.fc1 = nn.Linear(128*2*2, 500)
    self.dropout = nn.Dropout(0.7)
    self.fc2 = nn.Linear(500, 1)

  def forward(self, x):
    id= x

    id2 = self.pool(F.relu(self.conv_skip_con(id)))
    out = self.pool(F.relu(self.bn1(self.conv1(x))))
    out = self.pool(F.relu(self.bn2(self.conv2(out))))

    out = self.pool(F.relu(self.bn3(self.conv3(out))))
    out += id2
    out = out.view(-1, 128*2*2)
    out=self.dropout(F.relu(self.fc1(out)))
    out = F.sigmoid(self.fc2(out))
    return out

lr=1e-2
history = []
net = MyModule().to(device)
net.requires_grad_()
loss_fn=nn.CrossEntropyLoss()
criterion = nn.CrossEntropyLoss()
epochs = 10
lr=1e-2
history = []
net=net.to(device)
net.requires_grad_()
optimizer = optim.Adam(net.parameters(), lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=1.0)


In [ ]:
optimizer=torch.optim.Adam(net.parameters(), lr=0.00001)

def trainer(trainloader, val_loader, net):
  losses_train = []
  losses_val=[]
  accuracy_val=[]
  accuracy_train=[]
  for epoch in range(epochs):
      net.train()
      running_loss = 0
      correct = 0
      num = 0
      for i, (x_batch, y_batch) in enumerate(tqdm_notebook(trainloader)):
       # загружаем батч данных (вытянутый в линию)
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)

      # 2. вычисляем скор с помощью прямого распространения ( .forward or .__call__ )
        logits = net(x_batch)

    # 3. вычислеяем - функцию потерь (loss)
        loss=criterion(logits, y_batch)
        history.append(loss.item())
        running_loss+=loss.item()

    # 4. вычисляем градиенты
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        _, preds = torch.max(logits, axis=-1)
        correct += (preds == y_batch).sum().item()
        num += len(x_batch)
    # 5. шаг градиентного спуска
      losses_train.append(running_loss/len(trainloader))
      accuracy_train.append(correct / num)
      print("losses train:", losses_train)
      print("accuracy train: ", accuracy_train)
      net.eval()
      running_loss=0
      correct=0
      num=0
      with torch.no_grad():
        for xb, yb in tqdm_notebook(testloader):
          xb, yb = xb.to(device), yb.to(device)
          probs = net(xb)
          running_loss += criterion(probs, yb).item()

          _, preds = torch.max(probs, axis=-1)
          correct += (preds == yb).sum().item()
          num += len(xb)

      losses_val.append(running_loss / len(val_loader))
      accuracy_val.append(correct / num)
      print("losses validation:", losses_val)
      print('accuracy validation:', accuracy_val)
trainer(trainloader, testloader, net)

In [ ]:

gnet2 = models.googlenet(pretrained = True)


gnet2.fc = nn.Sequential(
    nn.Linear(1024, 1, bias = True),
    nn.Sigmoid()
)
gnet2.load_state_dict(torch.load('/content/drive/MyDrive/google_net_0.845_on20epoch_adam(0.0001), batch_size = 16 Imagenet.pth'))

gnet2 = gnet2.to(device)

class_correct = [0,0]
class_total = [0,0]
classes = ['Real', 'Fake']
gnet2.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)


        y_pred = gnet2(images)


        predicted = torch.squeeze(torch.round(y_pred))
        # print(predicted)
        # print(labels)
        c = predicted.cpu().detach() == labels


        #print(labels)
        for i in range(len(labels)):

            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print(class_correct, class_total)
print(sum(class_correct) / sum(class_total))


In [ ]:
resnet = models.resnet34(pretrained = True)
resnet.fc = nn.Sequential(
    nn.Linear(resnet.fc.in_features, 1),
    nn.Sigmoid()
)

resnet.load_state_dict(torch.load('/content/drive/MyDrive/resnet34_0.81_on20epoch_adam(0.0001), batch_size = 16 Imagenet.pth'))
#resnet.load_state_dict(torch.load('/content/own_0.745_on20epoch_adam(0.0001), batch_size = 16 Imagenet.pth'))


resnet = resnet.to(device)


class_correct = [0,0]
class_total = [0,0]
classes = ['Real', 'Fake']
resnet.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)


        y_pred = resnet(images)


        predicted = torch.squeeze(torch.round(y_pred))
        # print(predicted)
        # print(labels)
        c = predicted.cpu().detach() == labels


        #print(labels)
        for i in range(len(labels)):

            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print(class_correct, class_total)
print(sum(class_correct) / sum(class_total))


In [ ]:

effnet = models.efficientnet_b1(pretrained = True)


effnet.classifier[1] = nn.Sequential(
    nn.Linear(1280, 1),
    nn.Sigmoid()
)
effnet.load_state_dict(torch.load('/content/drive/MyDrive/effnet_0.88_on20epoch_adam(0.0001), batch_size = 16 Imagenet.pth'))

effnet = effnet.to(device)

class_correct = [0,0]
class_total = [0,0]
classes = ['Real', 'Fake']
effnet.eval()
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)


        y_pred = effnet(images)


        predicted = torch.squeeze(torch.round(y_pred))
        # print(predicted)
        # print(labels)
        c = predicted.cpu().detach() == labels


        #print(labels)
        for i in range(len(labels)):

            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

print(class_correct, class_total)
print(sum(class_correct) / sum(class_total))


In [ ]:
class FeatureExtractor(nn.Module):
    def __init__(self, original_model):
        super(FeatureExtractor, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-1])  # Remove last layer

    def forward(self, x):
        x = self.features(x)
        return x



class CombinedModel(nn.Module):
    def __init__(self, feature_dim):
        super(CombinedModel, self).__init__()
        # feature_dim is the total dimension of concatenated features
        self.classifier = nn.Sequential(
            nn.Linear(2816, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # Assuming x is the concatenated features
        x = self.classifier(x)
        return x

In [ ]:
def train_model(model, loss_fn, optimizer, scheduler = None, num_epochs = 10, effnet = False):

    fig = plt.figure(figsize=(10, 7))
    ax = fig.add_subplot(10, 1, 1)

    best_model_wts = model.state_dict()
    best_acc = 0.0
    acc = []



    for epoch in tqdm_notebook(range(num_epochs)):

        if scheduler is not None:
          scheduler.step()
        running_loss = 0.0
        model.train()

        for i, batch in enumerate(tqdm_notebook(trainloader)):

            X_batch, y_batch = batch
            y_batch = y_batch.float()
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()

            y_pred = model(X_batch)

            loss = loss_fn(y_pred, y_batch.unsqueeze(1))

            if not effnet:
              loss.backward()
            optimizer.step()

            running_loss += loss.item()

            if i % 25 == 24:
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 25))
                #losses.append(running_loss / 50)
                running_loss = 0.0


        class_correct = [0,0]
        class_total = [0,0]
        classes = ['Real', 'Fake']
        model.eval()

        with torch.no_grad():
            for data in testloader:
                images, labels = data
                images = images.to(device)

                y_pred = model(images)

                predicted = torch.squeeze(torch.round(y_pred))

                c = predicted.cpu().detach() == labels

                for i in range(len(labels)):
                    label = labels[i]
                    class_correct[label] += c[i].item()
                    class_total[label] += 1


        # for i in range(2):
        #     print("Accuracy of %5s : %2d %%" % (classes[i], 100 * class_correct[i] / class_total[i]))
        # print(class_total)
        # print(class_correct)
        print('Total Accuracy is: ', sum(class_correct) / sum(class_total))
        accuracy = sum(class_correct) / sum(class_total)
        acc.append( sum(class_correct) / sum(class_total))
        if accuracy > best_acc and accuracy > 0.72:
            best_acc = accuracy
            best_model_wts = model.state_dict()
            print("New Best Score!!! ")
            torch.save(model.state_dict(), 'own_' + str(accuracy) + '_on20epoch_adam(0.0001), batch_size = 16 Imagenet.pth')




    model.load_state_dict(best_model_wts)
    ax.plot(np.arange(len(acc)), acc)
    plt.show()
    return model, acc




In [ ]:
import gc
  # Delete variables or objects no longer needed
gc.collect()
torch.cuda.empty_cache()

In [ ]:


# Define feature extractors
extractor1 = FeatureExtractor(effnet)
extractor2 = FeatureExtractor(gnet2)
extractor3 = FeatureExtractor(resnet)

# Combined model

feature_dim = 1024 + 1280 + 1024
combined_model = CombinedModel(feature_dim)
combined_model = combined_model.to(device) # Set the correct feature_dim
combined_model.eval()
loss_fn = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(combined_model.parameters(), 0.001)

# Example training loop
# for i, (images, labels) in enumerate(tqdm_notebook(trainloader)):
#     images, label = images.to(device), labels.to(device)
#     labels = labels.float()
#     features1 = extractor1(images)
#     features2 = extractor2(images)
#     features3 = extractor3(images)

#     # Flatten and concatenate features
#     combined_features = torch.cat((features1.flatten(1), features2.flatten(1), features3.flatten(1)), dim=1)
#     # combined_features = combined_features.flatten()
#     #print(combined_features)

#     # Forward pass through the combined model
#     predictions = combined_model(combined_features)
#     predictions = predictions.to(device)

#     # Compute loss, backpropagate, and update
#     # print(predictions)
#     # print(labels)
#     loss = loss_fn(predictions.cpu().detach(), labels.unsqueeze(1))
#     optimizer.zero_grad()
#     #loss.backward()
#     optimizer.step()


fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(10, 1, 1)

best_model_wts = combined_model.state_dict()
best_acc = 0.0
acc = []



for epoch in tqdm_notebook(range(10)):

    running_loss = 0.0
    combined_model.train()

    for i, (images, labels) in enumerate(tqdm_notebook(trainloader)):
        # images, label = batch
        images, label = images.to(device), labels.to(device)
        labels = labels.float()
        #print(images)
        features1 = extractor1(images)
        features2 = extractor2(images)
        features3 = extractor3(images)

        # Flatten and concatenate features
        combined_features = torch.cat((features1.flatten(1), features2.flatten(1), features3.flatten(1)), dim=1)
        # combined_features = combined_features.flatten()
        #print(combined_features)

        # Forward pass through the combined model
        predictions = combined_model(combined_features)
        predictions = predictions.to(device)

        # Compute loss, backpropagate, and update
        # print(predictions)
        # print(labels)
        loss = loss_fn(predictions.cpu().detach(), labels.unsqueeze(1))
        optimizer.zero_grad()
        #loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 25 == 24:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 25))
            #losses.append(running_loss / 50)
            running_loss = 0.0


    class_correct = [0,0]
    class_total = [0,0]
    classes = ['Real', 'Fake']
    combined_model.eval()

    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)

            features1 = extractor1(images)
            features2 = extractor2(images)
            features3 = extractor3(images)


            combined_features = torch.cat((features1.flatten(1), features2.flatten(1), features3.flatten(1)), dim=1)

            # do the same with features
            y_pred = combined_model(combined_features) # combined images

            predicted = torch.squeeze(torch.round(y_pred))

            c = predicted.cpu().detach() == labels

            for i in range(len(labels)):
                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1


    # for i in range(2):
    #     print("Accuracy of %5s : %2d %%" % (classes[i], 100 * class_correct[i] / class_total[i]))
    # print(class_total)
    # print(class_correct)
    print('Total Accuracy is: ', sum(class_correct) / sum(class_total))
    accuracy = sum(class_correct) / sum(class_total)
    acc.append( sum(class_correct) / sum(class_total))
    if accuracy > best_acc and accuracy > 0.72:
        best_acc = accuracy
        best_model_wts = combined_model.state_dict()
        print("New Best Score!!! ")
        torch.save(combined_model.state_dict(), 'own_' + str(accuracy) + '_on20epoch_adam(0.0001), batch_size = 16 Imagenet.pth')




combined_model.load_state_dict(best_model_wts)
ax.plot(np.arange(len(acc)), acc)
plt.show()



In [ ]:

model1 = gnet2 #best for fake
model2 = resnet
model3 = effnet #best for real (close to 1)


weights = [0.5, 0, 0.5]


model1.eval()
model2.eval()
model3.eval()


class_correct = [0,0]
class_total = [0,0]
classes = ['Fake', 'Real']

with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)

        y_pred1 = model1(images)
        y_pred2 = model2(images)
        y_pred3 = model3(images)


        # y_pred = torch.empty(len(y_pred1))
        # for i in range(len(y_pred1)):       #85%
        #    if y_pred1[i] < (1 - y_pred3[i]):
        #       y_pred[i] = y_pred1[i]
        #    else:
        #       y_pred[i] = y_pred3[i]


        #y_pred = weights[0] * y_pred1 + weights[1] * y_pred2 + weights[2] * y_pred3     #89%


        y_pred = torch.empty(len(y_pred1))
        for i in range(len(y_pred1)):
          if y_pred3[i] > 0.5:
            y_pred[i] = y_pred3[i]
          elif y_pred1[i] < 0.5:
            y_pred[i] = y_pred1[i]
          else:
            y_pred = weights[0] * y_pred1 + weights[1] * y_pred2 + weights[2] * y_pred3



        predicted = torch.squeeze(torch.round(y_pred))

        c = predicted.cpu().detach() == labels

        for i in range(len(labels)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1

for i in range(2):
    print("Accuracy of %5s : %2d %%" % (classes[i], 100 * class_correct[i] / class_total[i]))
print(class_total)
print(class_correct)
print('Total Accuracy is: ', sum(class_correct) / sum(class_total))




In [ ]:
#check for what inputs have different results, check the intersection of snawers

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
teacher_model = resnet
student_model = ConvNet()

teacher_model = teacher_model.to(device)
student_model = student_model.to(device)

def train_knowledge_distillation(teacher, student, train_loader, epochs, learning_rate, T, soft_target_loss_weight, ce_loss_weight):
    ce_loss = torch.nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(student.parameters(), lr=learning_rate)

    teacher.eval()  # Teacher set to evaluation mode
    student.train() # Student to train mode

    for epoch in tqdm_notebook(range(epochs)):
        running_loss = 0.0
        for inputs, labels in tqdm_notebook(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            labels = labels.float()


            optimizer.zero_grad()

            # Forward pass with the teacher model - do not save gradients here as we do not change the teacher's weights
            with torch.no_grad():
                teacher_logits = teacher(inputs)

            # Forward pass with the student model
            student_logits = student(inputs)

            #Soften the student logits by applying softmax first and log() second
            soft_targets = nn.functional.softmax(teacher_logits / T, dim=-1)
            soft_prob = nn.functional.log_softmax(student_logits / T, dim=-1)

            # Calculate the soft targets loss. Scaled by T**2 as suggested by the authors of the paper "Distilling the knowledge in a neural network"
            soft_targets_loss = -torch.sum(soft_targets * soft_prob) / soft_prob.size()[0] * (T**2)

            # Calculate the true label loss
            label_loss = ce_loss(student_logits,  labels.unsqueeze(1))
            # predicted = torch.squeeze(torch.round(y_pred))

            # Weighted sum of the two losses
            # print(soft_targets_loss)
            # print(label_loss)
            loss = soft_target_loss_weight * soft_targets_loss + ce_loss_weight * label_loss

            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")
        class_correct = [0,0]
        class_total = [0,0]
        classes = ['Real', 'Fake']
        student_model.eval()
        with torch.no_grad():
            for data in testloader:
                images, labels = data
                images = images.to(device)

                y_pred = student(images)


                predicted = torch.squeeze(torch.round(y_pred))
                # print(predicted)
                # print(labels)
                c = predicted.cpu().detach() == labels


                #print(labels)
                for i in range(len(labels)):

                    label = labels[i]
                    class_correct[label] += c[i].item()
                    class_total[label] += 1

        print(class_correct, class_total)
        print(sum(class_correct) / sum(class_total))

# Apply ``train_knowledge_distillation`` with a temperature of 2. Arbitrarily set the weights to 0.75 for CE and 0.25 for distillation loss.
train_knowledge_distillation(teacher=teacher_model, student=student_model, train_loader=trainloader, epochs=10, learning_rate=0.001, T=2, soft_target_loss_weight=1, ce_loss_weight=1)


# Compare the student test accuracy with and without the teacher, after distillation
# print(f"Teacher accuracy: {test_accuracy_deep:.2f}%")
# print(f"Student accuracy without teacher: {test_accuracy_light_ce:.2f}%")
# print(f"Student accuracy with CE + KD: {test_accuracy_light_ce_and_kd:.2f}%")

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()
teacher_model = resnet
student_model = ConvNet()

teacher_model = teacher_model.to(device)
student_model = student_model.to(device)

student_model.train()
teacher_model.eval()

def distillation_loss(student_output, teacher_output, T=2.0):

    teacher_probs = torch.sigmoid(teacher_output / T)
    student_probs = torch.sigmoid(student_output / T)
    return nn.functional.mse_loss(student_probs, teacher_probs)

optimizer = optim.Adam(student_model.parameters(), lr=0.001)

In [ ]:

for epoch in tqdm_notebook(range(10)):
    student_model.train()
    running_loss = 0
    for i, (images, labels) in enumerate(tqdm_notebook(trainloader)):
        images, labels = images.to(device), labels.to(device)
        labels = labels.float()

        with torch.no_grad():
            teacher_output = teacher_model(images)

        student_output = student_model(images)

        true_loss = loss_fn(student_output, labels.unsqueeze(1))

        distill_loss = distillation_loss(student_output, teacher_output)

        loss = 0.5 * true_loss + 5 * distill_loss
        running_loss += loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 25 == 24:
          #print('On step: ', i, 'current loss: ', round(running_loss / 25, 2))
          #print(f'On step {i} current loss: {(running_loss / 25):.2f}')
          print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 25))
          running_loss = 0



    class_correct = [0,0]
    class_total = [0,0]
    classes = ['Real', 'Fake']
    student_model.eval()
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images = images.to(device)

            y_pred = student_model(images)


            predicted = torch.squeeze(torch.round(y_pred))
            # print(predicted)
            # print(labels)
            c = predicted.cpu().detach() == labels


            #print(labels)
            for i in range(len(labels)):

                label = labels[i]
                class_correct[label] += c[i].item()
                class_total[label] += 1

    print(class_correct, class_total)
    print(sum(class_correct) / sum(class_total))
